# Patient cells

In [ ]:
library(ggplot2)
library(extrafont)
library(stringr)
library(heatmaply)
library(ggrepel)
library(reticulate)
library(gridExtra)
library(robustbase)
library(randomForest)
library(reshape2)

In [ ]:
# Load external Python functions
source_python("reticulate_functions.py")
# Load extra fonts
ttf_import(paths = "/tmp/.fonts/")
loadfonts()
# Set this to true for faster compilation or false for more precise results and all outputs
TEST_MODE = F

In [ ]:
# Customize ggplot appearance

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
cust_pal = colorRampPalette(c("#008CAD", "#40B9D4", "#D4ECF2", "#F8B100", "#C00000", "#2D0000"))
yolla_pal = colorRampPalette(c('#FC7070', '#C00000', '#2D0000'))
scale_fill_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)

## Patient cells (different deficiencies)

In [ ]:
# Fix t-SNE layout and random number generation
set.seed(38)

In [ ]:
# NB: One row too much was imaged. 
# No cell is detected in these wells so 
#  they will be excluded of the analysis
LT = read.csv("Rsc/allImages_LT_patients.csv", header=T)

In [ ]:
getDonor <- function(row,column){
    if(row <= 5){
        if(column <= 7){
            return("ND1")
        } else if(column <= 11){
            return("PID_2466")
        } else if(column <= 15){
            return("ND2")
        } else {
            return("Was1")
        }
    } else if(row <= 7){
        if(column <= 7){
            return("PID_2378")
        } else if(column <= 11){
            return("ND3")
        } else if(column <= 15){
            return("PID_1076")
        } else {
            return("PID_2441")
        }
    } else if(row <= 9){            
        if(column <= 7){
            return("ND4")
        } else if(column <= 11){
            return("ARPC1B pt 1")
        } else if(column <= 15){
            return("ND5")
        } else {
            return("HD146")
        }
    } else if(row <= 7){            
        if(column <= 7){
            return("ND1")
        } else if(column <= 11){
            return("PID_2466")
        } else if(column <= 15){
            return("ND2")
        } else {
            return("Was1")
        }
    } else{
        if(column <= 7){
            return("WAST2")
        } else {
            return("ND6")
        }
    }        
}
getCoating <- function(column){
    return(ifelse(column%%2, "ICAM", "PLL"))
}
LT$Donor = mapply(getDonor, LT$Metadata_Row, LT$Metadata_Column)
LT$Coating = sapply(LT$Metadata_Column, getCoating)
# Is normal donor?
LT$ND = F
LT$ND[grep("ND", LT$Donor)] = T

We discard the data from the PLL coated plates and the non-ARPC1B deficient patients(not studied in this analysis).

In [ ]:
LT = LT[LT$Coating != "PLL",]
LT = LT[c(grep("ARPC1B",LT$Donor), grep("ND",LT$Donor)),]

In [ ]:
gpLT = ggplot(LT[!is.na(LT$Count_FilterNKNucleus),]) + geom_histogram(aes(Count_FilterNKNucleus, fill = as.factor(Metadata_Row)), binwidth=2) +
       scale_fill_discrete(name="Row")
gpLT
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_count.pdf", plot = gpLT)
}

In [ ]:
gpLT = ggplot(LT[!is.na(LT$Mean_FilterNKCytoplasm_AreaShape_Area),]) + geom_histogram(aes(Mean_FilterNKCytoplasm_AreaShape_Area, 
                                                                                    fill = as.factor(Donor)), binwidth=100) +
       scale_fill_discrete(name="Row")
gpLT
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_area.pdf", plot = gpLT)
}

### Filtering

In [ ]:
FILT_MAX_INT_DNA = 0.01 # Remove empty images and small DNA precipitations
FILT_MIN_CELLS = 5 # Most field of views have only 2 cells
FILT_NB_MAX_NA_IMAGE = 1
FILT_MAX_CORR = 0.6 # Keep uncorrelated variables

In [ ]:
ftToKeep = 1:dim(LT)[2]
# Make sure that the fields are numeric
ftToKeep <- ftToKeep[which(sapply(LT[,ftToKeep], class) == "numeric")]
# Remove execution time and count features
ftToKeep <- ftToKeep[grep("(Execution)|(Count)|(Concentration)|(ActinGranules)", colnames(LT)[ftToKeep], invert = T)]

In [ ]:
# Store all remaining features before filtering for downstream comparison to selected set
preFiltFt = colnames(LT)[ftToKeep]

In [ ]:
# Remove wells with low max DNA intensity
fieldToKeep <- which(LT$ImageQuality_MaxIntensity_DNA >= FILT_MAX_INT_DNA)
# Remove wells with low cell count
fieldToKeep <- na.omit(fieldToKeep[LT[fieldToKeep,]$Count_FilterNKNucleus >= FILT_MIN_CELLS])

In [ ]:
# Few bad quality pictures are generating a lot of missing values and are removed
fieldToKeep <- fieldToKeep[rowSums(is.na(LT[fieldToKeep,ftToKeep])) < FILT_NB_MAX_NA_IMAGE]

In [ ]:
# Remove remaining features with missing values
ftToKeep <- ftToKeep[colSums(is.na(LT[fieldToKeep,ftToKeep])) == 0] 
# Remove constant columns
indWT = LT[fieldToKeep,]$ND
ftToKeep <- ftToKeep[sapply(LT[fieldToKeep,ftToKeep], function(x) mad(x) != 0)]
ftToKeep <- ftToKeep[sapply(LT[fieldToKeep[indWT],ftToKeep], function(x) mad(x) != 0)]

In [ ]:
# Approximate normal distribution
transfLog <- function (x){
    minVal = min(x)
    return(log(x+1-minVal))
}

# Center and scale on control values
transfNorm <- function(x, y){
    return((x - median(y))/mad(y))
}

In [ ]:
# Try centering on all healthy donors
transformedLT = apply(LT[fieldToKeep, ftToKeep], 2, transfLog)
transformedLT = apply(transformedLT, 2, function(x) transfNorm(x, x[indWT]))

In [ ]:
# Order features from biggest mad to smallest mad
# Since features have mad(ND) = 1, it means that we rank features by how more variable they are
# for patients than for ND
orderFt = rev(order(apply(transformedLT, 2, mad)))

In [ ]:
uncorrFt = uncorrelate(transformedLT, orderCol = orderFt-1, threshold = FILT_MAX_CORR)
uncorrFt = unlist(uncorrFt) + 1

In [ ]:
transformedLT = transformedLT[,uncorrFt]

In [ ]:
dim(LT)
dim(LT[fieldToKeep, ftToKeep])
dim(transformedLT)

### Export subset of features

For LT cells, the following features are selected and explored separately for their biological interpretability:
```
* Actin intensity/cell (mean/well): LT$Intensity_MeanIntensity_CorrActin_FilterCytoplasm
* Cell area: LT$Mean_FilterCytoplasm_AreaShape_Area
* Cell roundness: LT$Mean_FilterCytoplasm_AreaShape_FormFactor
* Cell width: LT$Mean_FilterCytoplasm_AreaShape_MajorAxisLength 
* Cell length: LT$Mean_FilterCytoplasm_AreaShape_MinorAxisLength
* Cell length to width ratio: LT$Mean_FilterCytoplasm_AreaShape_MinorAxisLength / LT$Mean_FilterCytoplasm_AreaShape_MajorAxisLength
* Average number of perforin granules / cell: LT$Count_PerfGranules  / LT$Count_FilterCytoplasm
* Perforin area / cell area: (LT$Count_PerfGranules * LT$Mean_PerfGranules_AreaShape_Area)  / (LT$Count_FilterCytoplasm * LT$Mean_FilterCytoplasm_AreaShape_Area)
* Perforin intensity: LT$Intensity_MeanIntensity_CorrPerf_FilterCytoplasm
* Perforin area: LT$Mean_PerfGranules_AreaShape_Area
* Nucleus intensity: LT$Intensity_MeanIntensity_CorrDNA_FilteredNucleus
* Nucleus area: LT$Mean_FilteredNucleus_AreaShape_Area
* Nucleus roundness: LT$Mean_FilteredNucleus_AreaShape_FormFactor
* Nucleus width: LT$Mean_FilteredNucleus_AreaShape_MajorAxisLength 
* Nucleus length: LT$Mean_FilteredNucleus_AreaShape_MinorAxisLength
* Nucleus ratio: LT$Mean_FilteredNucleus_AreaShape_MinorAxisLength / LT$Mean_FilteredNucleus_AreaShape_MajorAxisLength
* Nucleus area / cell area: (LT$Count_FilteredNucleus * LT$Mean_FilteredNucleus_AreaShape_Area)  / (LT$Count_FilterCytoplasm * LT$Mean_FilterCytoplasm_AreaShape_Area)
* LFA intensity: LT$Intensity_MeanIntensity_CorrLFA_FilterNKLFA
* LFA-Actin correlation: LT[fieldToKeep,]$Mean_FilterNKCytoplasm_Correlation_Correlation_CorrLFA_CorrActin
* LFA area: LT$Mean_FilterNKLFA_AreaShape_Area
* LFA area / cell area: LT$Mean_FilterNKLFA_AreaShape_Area / (LT$Count_FilterCytoplasm * LT$Mean_FilterCytoplasm_AreaShape_Area)```

NB (from CellProfiler docs): FormFactor = $4 \times π \times Area / Perimeter^2$. Equals 1 for a perfectly circular object.

In [ ]:
subsetLT = cbind(Field = str_extract(as.character(LT[fieldToKeep,]$URL_Actin), "r..c..f.."),
                 Coating = as.character(LT[fieldToKeep,]$Coating),
                 Donor = as.character(LT[fieldToKeep,]$Donor),
                 ActinIntensity = LT[fieldToKeep,]$Intensity_MeanIntensity_CorrActin_FilterNKCytoplasm,
                 CellArea = LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_Area,
                 CellRoundness = LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_FormFactor,
                 CellWidth = LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_MajorAxisLength,
                 CellLength = LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_MinorAxisLength,
                 CellLengthOverWidth = LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_MinorAxisLength / 
                     LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_MajorAxisLength,
                 PerforinGranulesPerCell = LT[fieldToKeep,]$Count_FilterNKPerfGranules  / 
                     LT[fieldToKeep,]$Count_FilterNKCytoplasm,
                 PerforinAreaOverCellArea = (LT[fieldToKeep,]$Count_FilterNKPerfGranules * 
                     LT[fieldToKeep,]$Mean_PFilterNKPerfGranules_AreaShape_Area) / 
                     (LT[fieldToKeep,]$Count_FilterNKCytoplasm * 
                      LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_Area),
                 PerforinIntensity = LT[fieldToKeep,]$Intensity_MeanIntensity_CorrPerf_FilterNKCytoplasm,
                 PerforinArea = LT[fieldToKeep,]$Mean_FilterNKPerfGranules_AreaShape_Area,
                 NucleusIntensity = LT[fieldToKeep,]$Intensity_MeanIntensity_CorrDNA_FilterNKNucleus,
                 NucleusArea = LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_Area,
                 NucleusRoundness = LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_FormFactor,
                 NucleusWidth = LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_MajorAxisLength,
                 NucleusLength = LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_MinorAxisLength,
                 NucleusLengthOverWidth = LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_MinorAxisLength / 
                     LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_MajorAxisLength,
                 NucleusAreaOverCellArea = (LT[fieldToKeep,]$Count_FilterNKNucleus * 
                     LT[fieldToKeep,]$Mean_FilterNKNucleus_AreaShape_Area) / 
                     (LT[fieldToKeep,]$Count_FilterNKCytoplasm * 
                      LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_Area),
                 LfaIntensity = LT[fieldToKeep,]$Intensity_MeanIntensity_CorrLFA_FilterNKLFA,
                 LfaActinCorrelation = LT[fieldToKeep,]$Mean_FilterNKCytoplasm_Correlation_Correlation_CorrLFA_CorrActin,
                 LfaArea = LT[fieldToKeep,]$Mean_FilterNKLFA_AreaShape_Area,
                 LfaAreaOverCellArea = LT[fieldToKeep,]$Mean_FilterNKLFA_AreaShape_Area / 
                     LT[fieldToKeep,]$Mean_FilterNKCytoplasm_AreaShape_Area
                )

In [ ]:
SLT = as.data.frame(subsetLT)
SLT = SLT[SLT$Coating == "ICAM",]
colMeans(sapply(SLT[grep("ARPC1B",SLT$Donor),4:23], function(x) as.numeric(as.character(x)))) /
colMeans(sapply(SLT[grep("ND",SLT$Donor),4:23], function(x) as.numeric(as.character(x))))

In [ ]:
# Export list of "interpretable" features
if(!TEST_MODE){
    write.csv(subsetLT, "Tab/LT_Patients_features.csv", row.names = F)
}

In [ ]:
CountPerWell = aggregate(LT[fieldToKeep,]$Count_FilterNKCytoplasm, by = list(LT[fieldToKeep,]$Metadata_Well), 
                         FUN = function(x) sum(x, na.rm = T))
names(CountPerWell) <- c("Well", "Count")
CountPerCoating = aggregate(LT[fieldToKeep,]$Count_FilterNKCytoplasm, by = list(LT[fieldToKeep,]$Coating), 
                            FUN = function(x) sum(x, na.rm = T))
names(CountPerCoating) <- c("Coating", "Count")
CountPerDonor = aggregate(LT[fieldToKeep,]$Count_FilterNKCytoplasm, by = list(LT[fieldToKeep,]$Donor), 
                          FUN = function(x) sum(x, na.rm = T))
names(CountPerDonor) <- c("Donor", "Count")

In [ ]:
wellCountGranules = aggregate(LT[fieldToKeep,]$Count_FilterNKPerfGranules, 
                              by = list(LT[fieldToKeep,]$Metadata_Well), FUN = sum)
wellCountCells = aggregate(LT[fieldToKeep,]$Count_FilterNKCytoplasm, 
                           by = list(LT[fieldToKeep,]$Metadata_Well), FUN = sum)
stopifnot(wellCountGranules$Group.1 == wellCountCells$Group.1)
granulePerCellPerWell = data.frame(Well = wellCountGranules$Group.1, Average = wellCountGranules$x / wellCountCells$x)

In [ ]:
# Export cell counts
if(!TEST_MODE){
    write.csv(CountPerWell, "Tab/LT_Patients_count_well.csv", row.names = F)
    write.csv(CountPerCoating, "Tab/LT_Patients_count_coating.csv", row.names = F)
    write.csv(CountPerDonor, "Tab/LT_Patients_count_donor.csv", row.names = F)
    write.csv(granulePerCellPerWell, "Tab/LT_Patients_average_granule_count_per_cell.csv", row.names = F)
}

### Look at which types of features are kept

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf", "CorrLFA")
table(rowSums(sapply(catChannel, function(x) grepl(x, colnames(transformedLT)))))
dtCat = data.frame(CountIni = sort(colSums(sapply(catChannel, function(x) grepl(x, preFiltFt)))), 
                   Count = sort(colSums(sapply(catChannel, function(x) grepl(x, colnames(transformedLT))))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,700)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_SelecFt_Channel.pdf", plot = gp, width = 10)
}

In [ ]:
catObjects = c("FilterNKCytoplasm", "FilterNKLFA", "ShrunkenCytoplasm", "FilterNKNucleus", "FilterNKPerfGranules")
table(rowSums(sapply(catObjects, function(x) grepl(x, colnames(transformedLT)))))
dtCat = data.frame(CountIni = colSums(sapply(catObjects, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catObjects, function(x) grepl(x, colnames(transformedLT)))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,650)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_SelecFt_Object.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Threshold", "Granularity", "ImageQuality", "Texture", "Distance", "AreaShape", "RadialDistribution", "Neighbors", 
            "Correlation", "Intensity", "Overlap", "Location")
which(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedLT)))) == 0) # All features are covered

dtCat = data.frame(CountIni = colSums(sapply(catType, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catType, function(x) grepl(x, colnames(transformedLT)))))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_SelecFt_Type_All.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
table(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedLT)))))

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedLT))))
dtCount$Other = !apply(dtCount, 1, any)
dtCountIni = as.data.frame(sapply(catType, function(x) grepl(x, preFiltFt)))
dtCountIni$Other = !apply(dtCountIni, 1, any)
dtCat = data.frame(CountIni = colSums(dtCountIni), 
                   Count = colSums(dtCount))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_SelecFt_Type_Short.pdf", plot = gp, width = 10)
}

In [ ]:
# Export list of features kept
if(!TEST_MODE){
    write.table(colnames(transformedLT), file = "Tab/LT_Patients_list_features_kept.csv", col.names = F, row.names = F)
}

### Look at the morphological distribution of the fields of view

In [ ]:
# Fix random number generation
set.seed(38)

In [ ]:
umTLT = umap(transformedLT, min_dist = 0.1, neighbors = 10, n = 2, metric = "euclidean")
umTLT = as.data.frame(umTLT)
names(umTLT) = paste0("UMAP", 1:2)

In [ ]:
umTLT$Row <- as.factor(LT$Metadata_Row[fieldToKeep])
umTLT$Col <- as.factor(LT$Metadata_Column[fieldToKeep])
umTLT$URL <- as.factor(LT$URL_Actin[fieldToKeep])
umTLT$Coating <- as.factor(LT$Coating[fieldToKeep])
umTLT$Donor <- as.factor(LT$Donor[fieldToKeep])
umTLT$Well <- as.factor(LT$Metadata_Well[fieldToKeep])

In [ ]:
gp <- ggplot(umTLT) + geom_point(aes(UMAP1, UMAP2, color = Donor))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/LT_Patients_UMAP_all_donors.pdf", plot = gp, width = 10)
}

In [ ]:
umTLT$Count <- LT$Count_FilterNKNucleus[fieldToKeep]
gp <- ggplot(umTLT) + geom_point(aes(UMAP1, UMAP2, color = Count))
gp

In [ ]:
sessionInfo()